In [ ]:
pip install openai

##ユーザ要約のナゲット化（タスク複雑性低）

In [ ]:
import pandas as pd
from openai import OpenAI
import os
import re
from google.colab import userdata
from tqdm import tqdm

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

client = OpenAI()

df = pd.read_excel("your_file")

def extract_nuggets_from_output(text):
    return re.findall(r'\d+[\.．]?\s*(.+)', text)


def extract_nuggets(text: str):
    prompt = f"""
次の文章から「意味のある情報の最小単位（ナゲット）」を抽出してください。
ナゲットとは、対話システムがユーザに提供した意味のある情報の原子的，最小単位を指す．
また，主に，一文，またはそれ以下の情報の断片（句や節）である．
単独で再利用可能（=単独で意味が通じる）事実，主張，知識である．
必ず1行に1ナゲットずつ出力してください。

抽出のルール：
1.明確な意味を持つ情報断片（事実・定義・因果・例示など）を抽出対象とする．
2.ユーザの情報ニーズに直接または間接的に関連する内容を含める（補足情報もOK）．
3.1文に複数の独立した情報が含まれる場合は，ナゲットとして分割する。
4.意味が曖昧な主観語や儀礼的表現（例：「いいですね」「ありがとうございます」）は除外する．
5.例示・背景・文脈情報も，意味を持つ限り抽出対象とする．
6.すべてのナゲットは，可能な限り「単独で読んで理解できる形」で記述する．

例：
シナモンは料理やお菓子などに利用される香辛料で、原産地はスリランカや南インドです。
現在ではインドネシアやスリランカで多く生産されています。
また、最もシナモンの消費量が多い国はシナモンロールが有名なアメリカです。

例の抽出文：
1.シナモンは料理に利用される香辛料である
2.シナモンはお菓子に利用される香辛料である
3.原産地はスリランカである
4.原産地は南インドである
5.インドネシアで多く生産されている
6スリランカで多く生産されている
7.最もシナモンの消費量が多い国はアメリカである
8.アメリカはシナモンロールで有名である

文章：
{text}

出力フォーマット：
1. ～（ナゲット）
2. ～（ナゲット）
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4.1-2025-04-14",
            messages=[
                {"role": "system", "content": "あなたは，ナゲット抽出AIアシスタントです．抽出ルールに従い，必ず全IDに対してナゲットを抽出してください．"},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
        )
        raw_output = response.choices[0].message.content.strip()
        return extract_nuggets_from_output(raw_output)
    except Exception as e:
        return [f"[ERROR] {e}"]


# ナゲットの格納リスト
nugget_data = []

# tqdmで進捗表示、抜け対策も含める
for _, row in tqdm(df.iterrows(), total=len(df)):
    user_id = row["ID"]
    summary = row["Summary"]

    if pd.isnull(summary) or not str(summary).strip():
        continue  # 空Summaryはスキップ

    nuggets = extract_nuggets(summary)
    if not nuggets or "[ERROR]" in nuggets[0]:
        nugget_data.append({
            "ID": user_id,
            "NuggetID": f"{user_id}-0",
            "Nugget": "[抽出失敗]"
        })
        continue

    for i, nugget in enumerate(nuggets, 1):
        nugget_data.append({
            "ID": user_id,
            "NuggetID": f"{user_id}-{i}",
            "Nugget": nugget
        })

# データ保存
nugget_df = pd.DataFrame(nugget_data)
nugget_df.to_excel("your_file", index=False)
print("全IDに対して処理完了。抜け・エラーもログに残しました。")


##ユーザ要約のナゲット化（タスク複雑性高）

In [ ]:
import pandas as pd
from openai import OpenAI
import os
import re
from google.colab import userdata
from tqdm import tqdm

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

client = OpenAI()

df = pd.read_excel("your_file")

def extract_nuggets_from_output(text):
    return re.findall(r'\d+[\.．]?\s*(.+)', text)


def extract_nuggets(text: str):
    prompt = f"""
次の文章から「意味のある情報の最小単位（ナゲット）」を抽出してください。
ナゲットとは、対話システムがユーザに提供した意味のある情報の原子的，最小単位を指す．
また，主に，一文，またはそれ以下の情報の断片（句や節）である．
単独で再利用可能（=単独で意味が通じる）事実，主張，知識である．
必ず1行に1ナゲットずつ出力してください。


抽出のルール：
1.明確な意味を持つ情報断片（事実・定義・因果・例示など）を抽出対象とする．
2.ユーザの情報ニーズに直接または間接的に関連する内容を含める（補足情報もOK）．
3.1文に複数の独立した情報が含まれる場合は，ナゲットとして分割する。
4.意味が曖昧な主観語や儀礼的表現（例：「いいですね」「ありがとうございます」）は除外する．
5.例示・背景・文脈情報も，意味を持つ限り抽出対象とする．
6.すべてのナゲットは，可能な限り「単独で読んで理解できる形」で記述する．

例：
一人当たりのアルコール消費量比較は、国別ではチェコ、ドイツ、リトアニアなどヨーロッパのビール消費量の多い国が上位を占め、
少ない国は宗教上の理由で、イスラム圏の国は低い結果でした。日本の県別では人口比率が関係しており、
東京、大阪などの大都市圏があるところ、少ないのは佐賀、和歌山など人口の少ないところがあげられる。都市も人口比率が関係しているようです。

例の抽出文：
1.ビール消費量の多い国はチェコである
2.ビール消費量の多い国はドイツである
3.ビール消費量の多い国はリトアニアである
4.一人当たりのアルコール消費量比較は，ヨーロッパのビール消費量の多い国が上位を占める
5.少ない国は宗教上の理由である
6.イスラム圏の国は低い
7.日本の県別では人口比率が関係している
8.和歌山といった人口の少ないところが少ない
9.佐賀といった人口の少ないところが少ない
10.東京などの大都市圏があるところが多い
11.大阪などの大都市圏があるところが多い
12.都市も人口比率が関係している



文章：
{text}

出力フォーマット：
1. ～（ナゲット）
2. ～（ナゲット）
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4.1-2025-04-14",
            messages=[
                {"role": "system", "content": "あなたは，ナゲット抽出AIアシスタントです．抽出ルールに従い，必ず全IDに対してナゲットを抽出してください．"},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
        )
        raw_output = response.choices[0].message.content.strip()
        return extract_nuggets_from_output(raw_output)
    except Exception as e:
        return [f"[ERROR] {e}"]


# ナゲットの格納リスト
nugget_data = []


# tqdmで進捗表示、抜け対策も含める
for _, row in tqdm(df.iterrows(), total=len(df)):
    user_id = row["ID"]
    summary = row["Summary"]

    if pd.isnull(summary) or not str(summary).strip():
        continue  # 空Summaryはスキップ

    nuggets = extract_nuggets(summary)
    if not nuggets or "[ERROR]" in nuggets[0]:
        nugget_data.append({
            "ID": user_id,
            "NuggetID": f"{user_id}-0",
            "Nugget": "[抽出失敗]"
        })
        continue

    for i, nugget in enumerate(nuggets, 1):
        nugget_data.append({
            "ID": user_id,
            "NuggetID": f"{user_id}-{i}",
            "Nugget": nugget
        })

# データ保存
nugget_df = pd.DataFrame(nugget_data)
nugget_df.to_excel("your_file", index=False)
print("全IDに対して処理完了。抜け・エラーもログに残しました。")


##システム応答のナゲット化（タスク複雑性低）

In [ ]:
import pandas as pd
import re
from openai import OpenAI
from tqdm import tqdm
from google.colab import userdata
import os

# OpenAIクライアント初期化
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

client = OpenAI()

# 対話ペアのファイルを読み込む
df = pd.read_excel("your_file")

# ナゲット抽出関数
def extract_nuggets(text: str):
    prompt = f"""
次の文章から「意味のある情報の最小単位（ナゲット）」を抽出してください。
ナゲットとは、対話システムがユーザに提供した意味のある情報の原子的，最小単位を指す．
また，主に，一文，またはそれ以下の情報の断片（句や節）である．
単独で再利用可能（=単独で意味が通じる）事実，主張，知識である．
必ず1行に1ナゲットずつ出力してください。

抽出のルール：
1.明確な意味を持つ情報断片（事実・定義・因果・例示など）を抽出対象とする．
2.ユーザの情報ニーズに直接または間接的に関連する内容を含める（補足情報もOK）．
3.1文に複数の独立した情報が含まれる場合は，ナゲットとして分割する。
4.意味が曖昧な主観語や儀礼的表現（例：「いいですね」「ありがとうございます」）は除外する．
5.例示・背景・文脈情報も，意味を持つ限り抽出対象とする．
6.すべてのナゲットは，可能な限り「単独で読んで理解できる形」で記述する．

例1：
ヨーロッパで流通するビスケットに含まれるシナモンの原産地は、一般的にはカシアシナモンが使用されることが多いため、
中国、ベトナム、インドネシアのいずれかである可能性が高いです。
これらの国々はカシアシナモンの主要な生産国であり、リーズナブルな価格と豊かな風味から、商業用に広く利用されています。
一方、もしビスケットが高級志向のブランドや特に「セイロンシナモン」を使用すると明記されている場合、シナモンの原産地はスリランカである可能性が高まります。
この場合は、香りや風味がより繊細なのが特徴です。
具体的な原産地を知るためには、製品のパッケージや製造元の情報に記載されている場合もあるので、そちらを確認すると良いでしょう。

例1の抽出文：
1.ヨーロッパで流通するビスケットに含まれるシナモンの原産地は、一般的にはカシアシナモンが使用されることが多い
2.ヨーロッパで流通するビスケットに含まれるシナモンの原産地は中国の可能性が高い
3.ヨーロッパで流通するビスケットに含まれるシナモンの原産地はベトナムの可能性が高い
4.ヨーロッパで流通するビスケットに含まれるシナモンの原産地はインドネシアの可能性が高い
5.中国はカシアシナモンの主要な生産国である
6.ベトナムはカシアシナモンの主要な生産国である
7.インドネシアはカシアシナモンの主要な生産国である
6.カシアシナモンはリーズナブルな価格である
7.カシアシナモンは豊かな風味である
8.カシアシナモンは商業用に広く利用されている
9.ビスケットが高級志向のブランドや特に「セイロンシナモン」を使用すると明記されている場合、シナモンの原産地はスリランカである可能性がある
10.セイロンシナモンは香りがより繊細である
11.セイロンシナモンは風味がより繊細である
12.原産地を知るためには、製品のパッケージに記載されている
13.原産地を知るためには、製造元情報に記載されている

例2：
シナモンの原産地として主に挙げられる国は、スリランカです。
スリランカ産のシナモンは「セイロンシナモン」とも呼ばれ、高品質で香りが豊かです。
セイロンシナモンはスリランカのほか、インド南部やインドネシアの一部でも栽培されています。
また、一般的に「カシアシナモン」と呼ばれるシナモンは、インドネシア、中国、ベトナムなどで生産されています。
カシアシナモンはセイロンシナモンに比べて風味が強く、広く流通しているため、日常的に使用されることが多いです。
 これらの国々は、シナモンの主要な産地として知られており、世界中に供給されています。

例2の抽出文：
1.シナモンの原産地として主に挙げられる国は、スリランカである
2.スリランカ産のシナモンは「セイロンシナモン」とも呼ばれる
3.セイロンシナモンは高品質である
4.セイロンシナモンは香りが豊かである
5.セイロンシナモンはインド南部で栽培されている
6.セイロンシナモンはインドネシアの一部でも栽培されている
7.カシアシナモンはインドネシアで生産されている
8.カシアシナモンは中国で生産されている
9.カシアシナモンはベトナムで生産されている
10.カシアシナモンはセイロンシナモンに比べて風味が強い
11.カシアシナモンはセイロンシナモンに比べて広く流通している
12.カシアシナモンはセイロンシナモンに比べて日常的に使用されている
13.これらの国々は、シナモンの主要な産地として知られている
14.シナモンは世界中に供給されています


文章：
{text}

出力フォーマット：
1. ～（ナゲット）
2. ～（ナゲット）
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4.1-2025-04-14",
            messages=[
                {"role": "system", "content": "あなたは，ナゲット抽出AIアシスタントです．抽出ルールに従い，必ず全IDに対してナゲットを抽出してください．"},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        content = response.choices[0].message.content.strip()
        if "1." in content:
            content = content.split("1.", 1)[1]
            content = "1." + content
        nuggets = re.findall(r'\d+[\.．]?\s*(.+)', content)
        return nuggets
    except Exception as e:
        return [f"[ERROR] {e}"]

# 抽出ループ
nugget_records = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    pair_id = row["PairID"]
    user_id = row["ID"]
    user_query = row["UserQuery"]
    system_text = row["SystemText"]
    uuid_user = row["UUID_user"]
    uuid_bot = row["UUID_bot"]

    if pd.isnull(system_text) or not str(system_text).strip():
        nugget_records.append({
            "PairID": pair_id,
            "ID": user_id,
            "UserQuery": user_query,
            "NuggetID": f"{pair_id}-0",
            "Nugget": "[空応答]",
            "UUID_user": uuid_user,
            "UUID_bot": uuid_bot
        })
        continue

    nuggets = extract_nuggets(system_text)

    if not nuggets or "[ERROR]" in nuggets[0]:
        nugget_records.append({
            "PairID": pair_id,
            "ID": user_id,
            "UserQuery": user_query,
            "NuggetID": f"{pair_id}-0",
            "Nugget": "[抽出失敗]",
            "UUID_user": uuid_user,
            "UUID_bot": uuid_bot
        })
        continue

    for i, nugget in enumerate(nuggets, 1):
        nugget_records.append({
            "PairID": pair_id,
            "ID": user_id,
            "UserQuery": user_query,
            "SystemText": system_text,
            "NuggetID": f"{pair_id}-{i}",
            "Nugget": nugget,
            "UUID_user": uuid_user,
            "UUID_bot": uuid_bot
        })

# 保存
nugget_df = pd.DataFrame(nugget_records)
nugget_df.to_excel("your_file", index=False)
print("✅ ナゲット抽出完了：your_file に保存しました")


##システム応答のナゲット化（タスク複雑性高）

In [ ]:
import pandas as pd
import re
from openai import OpenAI
from tqdm import tqdm
import os

# OpenAIクライアント初期化
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

client = OpenAI()

# 対話ペアのファイルを読み込む
df = pd.read_excel("your_file")

# ナゲット抽出関数
def extract_nuggets(text: str):
    prompt = f"""
次の文章から「意味のある情報の最小単位（ナゲット）」を抽出してください。
ナゲットとは、対話システムがユーザに提供した意味のある情報の原子的，最小単位を指す．
また，主に，一文，またはそれ以下の情報の断片（句や節）である．
単独で再利用可能（=単独で意味が通じる）事実，主張，知識である．
必ず1行に1ナゲットずつ出力してください。

抽出のルール：
1.明確な意味を持つ情報断片（事実・定義・因果・例示など）を抽出対象とする．
2.ユーザの情報ニーズに直接または間接的に関連する内容を含める（補足情報もOK）．
3.1文に複数の独立した情報が含まれる場合は，ナゲットとして分割する。
4.意味が曖昧な主観語や儀礼的表現（例：「いいですね」「ありがとうございます」）は除外する．
5.例示・背景・文脈情報も，意味を持つ限り抽出対象とする．
6.すべてのナゲットは，可能な限り「単独で読んで理解できる形」で記述する．

例1：
最新のデータによると、日本人の一人当たりのアルコール消費量は、年間約7～8リットルの純アルコールに相当します。
この数値は成人（15歳以上）を対象にしており、ビール、ワイン、スピリッツ、日本酒などを含むすべてのアルコール飲料から計算されています。
この消費量は、世界の他の国々と比較すると中間に位置しています。
アルコール消費の文化や習慣は国や地域によって異なるため、この数値はそれぞれの文化的背景や健康政策の影響を受けています。
日本ではビールや日本酒が人気ですが、最近ではワインやハイボールなどのスピリッツの消費も増えています。
これに伴い、適度な飲酒を心掛け、健康に配慮した飲酒習慣を推進する努力が続けられています。
具体的な消費量は毎年変動する可能性があるため、最新の統計情報を知りたい場合は、厚生労働省や酒類総合研究所などの公式なデータを参照することをお勧めします。

例1の抽出文：
1.日本人の一人当たりのアルコール消費量は、年間約7～8リットルの純アルコールに相当する
2.この数値は成人（15歳以上）を対象としている
3.ビール、ワイン、スピリッツ、日本酒などを含むすべてのアルコール飲料から計算している
4.この消費量は、世界の他の国々と比較すると中間に位置している
5.アルコール消費の文化や習慣は国によって異なる
6.アルコール消費の文化や習慣は地域によって異なる
7.それぞれの文化的背景の影響を受けている
8.それぞれの健康政策の影響を受けている
9.日本ではビールが人気である
10.日本では日本酒が人気である
11.最近ではワインなどのスピリッツの消費も増えている
12.最近ではハイボールなどのスピリッツの消費も増えている
13.適度な飲酒を心掛け、健康に配慮した飲酒習慣を推進する努力が続けられている
14.具体的な消費量は毎年変動する可能性がある
15.最新の統計情報を知りたい場合は、酒類総合研究所などの公式なデータを参照することをお勧めする
16.最新の統計情報を知りたい場合は、厚生労働省など公式なデータを参照することをお勧めする

例2：
1人当たりのアルコール消費量は年齢によって違うことがあります。
以下にその理由と傾向を示します： 1. **若年層**: 20歳から30歳の若年層では、社会に出てからの付き合いや飲み会などでアルコールを飲む機会が増えることがあります。
しかし、近年では若者の「飲酒離れ」も進んでおり、健康志向やライフスタイルの変化によって、必ずしも大きな消費量にはつながっていない場合もあります。
2. **中年層**: 30代から50代の中年層は、仕事の付き合いや家庭内での食事の一部としてアルコールを摂取する機会が比較的多い傾向があります。
この層では、ストレス解消としてアルコールを飲む人も一定数います。
3. **高齢層**: 高齢層では、健康を意識した飲酒習慣が形成されることが多く、量よりも質を重視する傾向があります。
ただし、長年の習慣で飲み続けている場合もあるので、個人差があります。
年齢とともに飲酒に対する意識や習慣が変わるため、年齢層ごとの消費量には違いが見られるのです。
また、健康状態、社会的役割の変化、ライフスタイルなどによっても影響を受けるため、同じ年齢層でも個々の状況に応じて消費量は異なることがあります。
これらの傾向は、地域社会や文化によっても多少異なる可能性があります。

例2の抽出文：
1.1人当たりのアルコール消費量は年齢によって違う
2.20歳から30歳の若年層では、社会に出てからの付き合いでアルコールを飲む機会が増えることがある
3.20歳から30歳の若年層では、社会に出てから飲み会などでアルコールを飲む機会が増えることがある
4.近年では若者の「飲酒離れ」も進んでいる
5.健康志向変化によって、必ずしも大きな消費量にはつながっていない場合がある．
6.ライフスタイルの変化によって、必ずしも大きな消費量にはつながっていない場合がある
7. 30代から50代の中年層は、仕事の付き合いとしてアルコールを摂取する機会が比較的多い傾向にある
8.30代から50代の中年層は、家庭内での食事の一部としてアルコールを摂取する機会が比較的多い傾向にある
9.ストレス解消としてアルコールを飲む人も一定数います
10.高齢層では、健康を意識した飲酒習慣が形成されることが多い
11.量よりも質を重視する傾向がある
12.長年の習慣で飲み続けている場合もあるので、個人差がある
13.年齢とともに飲酒に対する意識が変わる
14.年齢とともに飲酒に対する習慣が変わる
15.年齢層ごとの消費量には違いが見られる
16.健康状態によっても影響を受ける
17.社会的役割の変化によっても影響を受ける
18.ライフスタイルなどによっても影響を受ける
19.同じ年齢層でも個々の状況に応じて消費量は異なる
20.これらの傾向は、地域社会によっても多少異なる可能性がある

文章：
{text}

出力フォーマット：
1. ～（ナゲット）
2. ～（ナゲット）
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4.1-2025-04-14",
            messages=[
                {"role": "system", "content": "あなたは，ナゲット抽出AIアシスタントです．抽出ルールに従い，必ず全IDに対してナゲットを抽出してください．"},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        content = response.choices[0].message.content.strip()
        if "1." in content:
            content = content.split("1.", 1)[1]
            content = "1." + content
        nuggets = re.findall(r'\d+[\.．]?\s*(.+)', content)
        return nuggets
    except Exception as e:
        return [f"[ERROR] {e}"]

# 抽出ループ
nugget_records = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    pair_id = row["PairID"]
    user_id = row["ID"]
    user_query = row["UserQuery"]
    system_text = row["SystemText"]
    uuid_user = row["UUID_user"]
    uuid_bot = row["UUID_bot"]

    if pd.isnull(system_text) or not str(system_text).strip():
        nugget_records.append({
            "PairID": pair_id,
            "ID": user_id,
            "UserQuery": user_query,
            "NuggetID": f"{pair_id}-0",
            "Nugget": "[空応答]",
            "UUID_user": uuid_user,
            "UUID_bot": uuid_bot
        })
        continue

    nuggets = extract_nuggets(system_text)

    if not nuggets or "[ERROR]" in nuggets[0]:
        nugget_records.append({
            "PairID": pair_id,
            "ID": user_id,
            "UserQuery": user_query,
            "NuggetID": f"{pair_id}-0",
            "Nugget": "[抽出失敗]",
            "UUID_user": uuid_user,
            "UUID_bot": uuid_bot
        })
        continue

    for i, nugget in enumerate(nuggets, 1):
        nugget_records.append({
            "PairID": pair_id,
            "ID": user_id,
            "UserQuery": user_query,
            "SystemText": system_text,
            "NuggetID": f"{pair_id}-{i}",
            "Nugget": nugget,
            "UUID_user": uuid_user,
            "UUID_bot": uuid_bot
        })

# 保存
nugget_df = pd.DataFrame(nugget_records)
nugget_df.to_excel("your_file", index=False)
print("✅ ナゲット抽出完了：your_file に保存しました")